# Algoritmo de Regressão Linear com XGboost
<hr>

## Importando Pacotes

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore') 

## Carregando Dados

In [ ]:
df_mpg = sns.load_dataset('mpg')

In [ ]:
df_mpg.head()

In [ ]:
print(f'Quantidade de Linhas: {df_mpg.shape[0]}\nQuantidade de Colunas: {df_mpg.shape[1]}')

## Análise Exploratória

In [ ]:
df_mpg.describe().T

## Separando as bases

In [ ]:
x_mpg = df_mpg.drop(['mpg','origin','name'], axis=1)
y_mpg = df_mpg['mpg']

In [ ]:
x_mpg_train, x_mpg_test, y_mpg_train, y_mpg_test = train_test_split(x_mpg, y_mpg, random_state=42)

In [ ]:
print(f'Quantidade de dados para treino: {x_mpg_train.shape[0]}\nQuantidade de dados para teste: {x_mpg_test.shape[0]}\nTotal: {x_mpg_train.shape[0] + x_mpg_test.shape[0]}')
print(f'\nPorcentagem de dados para teste: {len(x_mpg_test) / len(x_mpg) *100:.2f}%')

## Avaliando Modelos

In [ ]:
regressor_xgb = xgb.XGBRegressor()

resultado_cv = cross_val_score(regressor_xgb, x_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error')
print(f'\nResultado da avaliação: {resultado_cv}\nErro Médio: {resultado_cv.mean():.2f}')

In [ ]:
regressor_xgb_tunado = xgb.XGBRegressor(max_depth=2)

resultado_cv_tunado = cross_val_score(regressor_xgb_tunado, x_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error')
print(f'\nResultado da avaliação: {resultado_cv_tunado}\nErro: {resultado_cv_tunado.mean():.2f}')

In [ ]:
regressor_xgb_gblinear = xgb.XGBRegressor(booster='gblinear')

resultado_cv_gblinear = cross_val_score(regressor_xgb_gblinear, x_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error')
print(f'\nResultado da avaliação: {resultado_cv_gblinear}\nErro: {resultado_cv_gblinear.mean():.2f}')

## Modelo Campeão

In [ ]:
# rmsle se assemelha ao neg root mean squares error
regressor_campeao = xgb.XGBRegressor(eval_metric='rmsle')
print(f'Objetivo: {regressor_campeao.objective}\nTipo: {type(regressor_campeao)}')

## Treino

In [ ]:
regressor_campeao.fit(x_mpg_train, y_mpg_train)

## Teste

In [ ]:
predicoes_mpg = regressor_campeao.predict(x_mpg_test)

## Predição

In [ ]:
print(f'Predições:\n{predicoes_mpg[:10]}\n\nBase de Teste:\n{y_mpg_test[:10]}')

In [ ]:
print(f'Acurácia: {regressor_campeao.score(x_mpg_test, y_mpg_test)*100:.2f}')

## Avaliação do Modelo

In [ ]:
# Para avaliarmos a qualidade do modelo de Regressão, usualmente utilizamos o RMSE (Raiz do Erro Quadrático Médio). 
# O RMSE representa a distância média entre as previsões e a linha de regressão. 
# Quanto maior o RMSE do modelo, menor é o seu desempenho, já que valores altos indicam maior quantidade de erros de predição.

mse = mean_squared_error(y_mpg_test, predicoes_mpg)
rmse = math.sqrt(mse)
print(f'Erro quadrático médio: {mse:.2f}\nRaiz do erro quadrático médio: {rmse:.2f}')

## Importância das Features

In [ ]:
regressor_campeao.feature_importances_
feature_importances = pd.DataFrame(regressor_campeao.feature_importances_,
                                   index = x_mpg_train.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

## Salvando Dados

In [ ]:
df  =  x_mpg_test
df['CombustivelPredito'] = predicoes_mpg

In [ ]:
df.to_csv('./output/01_mpg_regression_Model.csv', sep=';', index=False)

## Salvando o Modelo

In [ ]:
joblib.dump(regressor_campeao, './model/01_mpg_regression_Model.pkl')

## Testando o Modelo

In [ ]:
loaded_model = joblib.load('./model/01_mpg_regression_Model.pkl')
new_data = pd.read_csv('./data/01_mpg_regression_test_newdata.csv', delimiter=';')

In [ ]:
result = loaded_model.predict(new_data)
df_new, df_new['CombustivelPredito']= new_data, result
df_new